In [2]:
%pylab inline
import numpy as np
from tqdm import trange

Populating the interactive namespace from numpy and matplotlib


In [14]:
def fetch(url):
    import os, hashlib, requests, gzip
    fp = os.path.join("/home/klaas/Projects/Mnist", hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            dat = f.read()
    else:
        with open(fp, "wb") as f:
            dat = requests.get(url).content
            f.write(dat)
    return numpy.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()
X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

In [20]:
X_train[0:10].shape

(10, 28, 28)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.set_printoptions(sci_mode=False)

class BobNet(nn.Module):
    def __init__(self):
        super(BobNet, self).__init__()
        self.l1 = nn.Linear(784, 128, bias=False)
        self.l2 = nn.Linear(128, 10, bias=False)
        self.sm = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        x = self.sm(x)
        return x

In [66]:
model = BobNet()
loss_function = nn.NLLLoss(reduction='none')
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0)
BS = 32
losses, accuracies = [], []

for i in (t := trange (1000)):
    samp = np.random.randint(0, X_train.shape[0], size=(BS))
    X = torch.Tensor(X_train[samp].reshape((-1, 28*28))).float()
    Y = torch.Tensor(Y_train[samp]).long()
    model.zero_grad()
    
    out = model(X)
    cat = torch.argmax(out, dim=1)
    accuracy = (cat == Y).float().mean()
    loss = loss_function(out, Y)
    loss = loss.mean()
    loss.backward()
    optim.step()
    
    loss, accuracy = loss.item(), accuracy.item()

100%|██████████| 2/2 [00:00<00:00, 1437.14it/s]


In [74]:
loss_function(out, Y).mean().item()


32.14827346801758

In [71]:
cat

tensor([4, 4, 4, 7, 1, 4, 4, 4, 1, 4, 7, 1, 7, 1, 5, 4, 7, 4, 7, 4, 1, 7, 4, 4,
        1, 1, 4, 9, 9, 7, 1, 1])

In [72]:
Y

tensor([1, 1, 4, 1, 6, 9, 1, 8, 7, 1, 9, 5, 8, 3, 0, 1, 7, 7, 3, 7, 5, 5, 6, 3,
        6, 3, 8, 6, 1, 7, 7, 6])